## Dependencies

In [22]:
# Confirmed needed dependencies
import random
import pprint
import pandas as pd
import requests
import json

# Dependencies for geocoordinates generator
import sys
import math
import gmplot

# Dependencies for conversion of coordinates to addresses
import geopy
from geopy.geocoders import Nominatim
import time

# Dependencies for Zillow data
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults

# Dependency for Heat Mapper
import gmaps


# Add config.py file with the following variables and cooresponding Zillow API keys
# from config import Ztroy, Zseth, Zkat, Zval, Zyuta
# from config import google_API_Key


################# ONGOING EDITS TO REQUIREMENTS.MD #################
###### IF ANY ERRORS OCCUR RELATING TO MODULES OR CONFIG.PY #######
### REFER TO requirements.md TO ENSURE YOU ARE PROPERLY SETUP ####

## File inputs/outputs

In [2]:
randLatLon_csv = "./Data/randomLatLon.csv" 
addressList_csv = "./Data/addressList.csv"
masterData_csv = "./Data/masterData.csv"

## Geocoordinates of Austin

In [ ]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

In [3]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 20         #### Set radius in miles

points = 40000        #### Set number of lat,lon points to generate

In [4]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points += 1

In [5]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

len(lat_lon_df)

40000

In [6]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(randLatLon_csv, index=False, header=True)

## Plot coordinate points on map


In [17]:
# This section will plot points on a Google map centered at centerPointLat and centerPointLon with a magnification of magFactor
# It assumes there is a dataframe with "lat" and "lon" columns
# The resulting map is saved to a file called 

centerPointLat = 30.27444  #these are the coordinates of the Texas State Capitol building
centerPointLon = -97.74028 #these are the coordinates of the Texas State Capitol building
magnificationFactor = 10
pointColor = "red"
pointSize = 100
mapOutputFile = "mymap.html"
df = lat_lon_df

gmap = gmplot.GoogleMapPlotter(centerPointLat, centerPointLon, magnificationFactor)

gmap.scatter(df["lat"], df["lon"], pointColor, size=pointSize, marker=False)

gmap.draw("./Visuals/" + mapOutputFile)

## Convert Coordinates to Residential Addresses

In [7]:
#########################
##### Yuta's Blocks #####
#########################

##### Geopy Nominatim API #####
geopy.geocoders.options.default_user_agent = "ut-group-EPIC"

pp = pprint.PrettyPrinter(indent=4)

url = "https://nominatim.openstreetmap.org/reverse?"

In [8]:
# Test API - Known Residential Address
params_1 = {
    "format": "jsonv2",
    "lat": 30.440777,
    "lon": -97.777048
}

print("===== Test Home Response:")
response = requests.get(url, params=params_1).json()
pp.pprint(response)
print("\n" + "="*60 + "\n")

===== Test Home Response:
{   'address': {   'country': 'USA',
                   'country_code': 'us',
                   'county': 'Williamson County',
                   'hamlet': 'Pond Springs',
                   'house_number': '12126',
                   'neighbourhood': "Hunter's Chase",
                   'postcode': '78729',
                   'road': 'Hunters Chase Drive',
                   'state': 'Texas',
                   'suburb': 'Jollyville'},
    'addresstype': 'place',
    'boundingbox': ['30.440962', '30.441162', '-97.777478', '-97.777278'],
    'category': 'place',
    'display_name': "12126, Hunters Chase Drive, Hunter's Chase, Jollyville, "
                    'Pond Springs, Williamson County, Texas, 78729, USA',
    'importance': '0',
    'lat': '30.441062',
    'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
               'https://osm.org/copyright',
    'lon': '-97.777378',
    'name': None,
    'osm_id': '3825454876',
    'osm_type': 'node',
  

In [9]:
# Import CSV, put into DataFrame
latlon_df = pd.read_csv(randLatLon_csv)
latlon_df.head()

,lat,lon
0,30.015745,-97.820335
1,30.311626,-97.947056
2,30.160044,-97.969449
3,30.166979,-97.533584
4,30.121553,-97.792665


In [10]:
# Put latitudes and longitudes into a zip object
lats = latlon_df.iloc[:, 0]
lons = latlon_df.iloc[:, 1]
lat_lons = []
lat_lons = zip(lats, lons)

In [12]:
##### Loop Request API for Addresses / Append to lists #####
# Make sure to import time

query_url = "https://nominatim.openstreetmap.org/reverse?"

house_num = []
road = []
postcode = []
lat = []
lon = []
neighborhood = []

counter = 1
numRequests = latlon_df["lat"].count()
rSuccess = []
rFailure = []

print(f"Processing {numRequests} Requests...")

# Nominatim API Request

for lat_lon in lat_lons:
    params = {
        "format": "jsonv2",
        "lat": lat_lon[0],
        "lon": lat_lon[1]
    }

    time.sleep(1.1)
    response = requests.get(query_url, params=params).json()

    if response['type'] == 'house' or response['type'] == 'yes':
        lat.append(response['lat'])
        lon.append(response['lon'])
        
        try:
            postcode.append(response['address']['postcode'])
        except (KeyError, IndexError):
            postcode.append("NA")
        try:
            house_num.append(response['address']['house_number'])
        except (KeyError, IndexError):
            house_num.append("NA")
        try:
            road.append(response['address']['road'])
        except (KeyError, IndexError):
            road.append("NA")
        try:
            neighborhood.append(response['address']['neighbourhood'])
        except (KeyError, IndexError):
            neighborhood.append("NA")
        
        print(f"Processed Record {counter} of {numRequests}.")
        rSuccess.append(counter)
        counter += 1
        
    else:
        print(f"Wrong Type - Skipped Record {counter} of {numRequests}.")
        rFailure.append(counter)
        counter += 1
        
print(f"Finished Requests !!!")

Processing 40000 Requests...
Processed Record 1 of 40000.
Processed Record 2 of 40000.
Processed Record 3 of 40000.
Processed Record 4 of 40000.
Wrong Type - Skipped Record 5 of 40000.
Wrong Type - Skipped Record 6 of 40000.
Processed Record 7 of 40000.
Wrong Type - Skipped Record 8 of 40000.
Wrong Type - Skipped Record 9 of 40000.
Processed Record 10 of 40000.
Processed Record 11 of 40000.
Processed Record 12 of 40000.
Wrong Type - Skipped Record 13 of 40000.
Processed Record 14 of 40000.
Processed Record 15 of 40000.
Wrong Type - Skipped Record 16 of 40000.
Wrong Type - Skipped Record 17 of 40000.
Processed Record 18 of 40000.
Wrong Type - Skipped Record 19 of 40000.
Processed Record 20 of 40000.
Wrong Type - Skipped Record 21 of 40000.
Processed Record 22 of 40000.
Processed Record 23 of 40000.
Wrong Type - Skipped Record 24 of 40000.
Wrong Type - Skipped Record 25 of 40000.
Wrong Type - Skipped Record 26 of 40000.
Wrong Type - Skipped Record 27 of 40000.
Wrong Type - Skipped Record

Processed Record 232 of 40000.
Wrong Type - Skipped Record 233 of 40000.
Processed Record 234 of 40000.
Wrong Type - Skipped Record 235 of 40000.
Processed Record 236 of 40000.
Wrong Type - Skipped Record 237 of 40000.
Processed Record 238 of 40000.
Processed Record 239 of 40000.
Wrong Type - Skipped Record 240 of 40000.
Wrong Type - Skipped Record 241 of 40000.
Wrong Type - Skipped Record 242 of 40000.
Wrong Type - Skipped Record 243 of 40000.
Processed Record 244 of 40000.
Wrong Type - Skipped Record 245 of 40000.
Wrong Type - Skipped Record 246 of 40000.
Processed Record 247 of 40000.
Processed Record 248 of 40000.
Processed Record 249 of 40000.
Wrong Type - Skipped Record 250 of 40000.
Processed Record 251 of 40000.
Processed Record 252 of 40000.
Wrong Type - Skipped Record 253 of 40000.
Processed Record 254 of 40000.
Wrong Type - Skipped Record 255 of 40000.
Processed Record 256 of 40000.
Wrong Type - Skipped Record 257 of 40000.
Wrong Type - Skipped Record 258 of 40000.
Wrong Typ

Wrong Type - Skipped Record 464 of 40000.
Processed Record 465 of 40000.
Processed Record 466 of 40000.
Wrong Type - Skipped Record 467 of 40000.
Processed Record 468 of 40000.
Processed Record 469 of 40000.
Processed Record 470 of 40000.
Processed Record 471 of 40000.
Processed Record 472 of 40000.
Wrong Type - Skipped Record 473 of 40000.
Processed Record 474 of 40000.
Wrong Type - Skipped Record 475 of 40000.
Wrong Type - Skipped Record 476 of 40000.
Processed Record 477 of 40000.
Processed Record 478 of 40000.
Wrong Type - Skipped Record 479 of 40000.
Processed Record 480 of 40000.
Wrong Type - Skipped Record 481 of 40000.
Wrong Type - Skipped Record 482 of 40000.
Processed Record 483 of 40000.
Processed Record 484 of 40000.
Wrong Type - Skipped Record 485 of 40000.
Wrong Type - Skipped Record 486 of 40000.
Wrong Type - Skipped Record 487 of 40000.
Wrong Type - Skipped Record 488 of 40000.
Wrong Type - Skipped Record 489 of 40000.
Processed Record 490 of 40000.
Wrong Type - Skipped

Processed Record 692 of 40000.
Wrong Type - Skipped Record 693 of 40000.
Processed Record 694 of 40000.
Wrong Type - Skipped Record 695 of 40000.
Processed Record 696 of 40000.
Processed Record 697 of 40000.
Wrong Type - Skipped Record 698 of 40000.
Wrong Type - Skipped Record 699 of 40000.
Wrong Type - Skipped Record 700 of 40000.
Wrong Type - Skipped Record 701 of 40000.
Processed Record 702 of 40000.
Wrong Type - Skipped Record 703 of 40000.
Wrong Type - Skipped Record 704 of 40000.
Processed Record 705 of 40000.
Wrong Type - Skipped Record 706 of 40000.
Wrong Type - Skipped Record 707 of 40000.
Wrong Type - Skipped Record 708 of 40000.
Processed Record 709 of 40000.
Processed Record 710 of 40000.
Wrong Type - Skipped Record 711 of 40000.
Wrong Type - Skipped Record 712 of 40000.
Processed Record 713 of 40000.
Processed Record 714 of 40000.
Processed Record 715 of 40000.
Processed Record 716 of 40000.
Processed Record 717 of 40000.
Wrong Type - Skipped Record 718 of 40000.
Processed

Processed Record 921 of 40000.
Wrong Type - Skipped Record 922 of 40000.
Processed Record 923 of 40000.
Wrong Type - Skipped Record 924 of 40000.
Processed Record 925 of 40000.
Processed Record 926 of 40000.
Wrong Type - Skipped Record 927 of 40000.
Wrong Type - Skipped Record 928 of 40000.
Wrong Type - Skipped Record 929 of 40000.
Wrong Type - Skipped Record 930 of 40000.
Wrong Type - Skipped Record 931 of 40000.
Processed Record 932 of 40000.
Wrong Type - Skipped Record 933 of 40000.
Wrong Type - Skipped Record 934 of 40000.
Processed Record 935 of 40000.
Wrong Type - Skipped Record 936 of 40000.
Wrong Type - Skipped Record 937 of 40000.
Wrong Type - Skipped Record 938 of 40000.
Wrong Type - Skipped Record 939 of 40000.
Wrong Type - Skipped Record 940 of 40000.
Wrong Type - Skipped Record 941 of 40000.
Processed Record 942 of 40000.
Wrong Type - Skipped Record 943 of 40000.
Processed Record 944 of 40000.
Wrong Type - Skipped Record 945 of 40000.
Wrong Type - Skipped Record 946 of 400

...

Wrong Type - Skipped Record 39015 of 40000.
Wrong Type - Skipped Record 39016 of 40000.
Wrong Type - Skipped Record 39017 of 40000.
Wrong Type - Skipped Record 39018 of 40000.
Processed Record 39019 of 40000.
Wrong Type - Skipped Record 39020 of 40000.
Processed Record 39021 of 40000.
Processed Record 39022 of 40000.
Wrong Type - Skipped Record 39023 of 40000.
Wrong Type - Skipped Record 39024 of 40000.
Processed Record 39025 of 40000.
Processed Record 39026 of 40000.
Processed Record 39027 of 40000.
Processed Record 39028 of 40000.
Wrong Type - Skipped Record 39029 of 40000.
Processed Record 39030 of 40000.
Wrong Type - Skipped Record 39031 of 40000.
Processed Record 39032 of 40000.
Processed Record 39033 of 40000.
Wrong Type - Skipped Record 39034 of 40000.
Processed Record 39035 of 40000.
Processed Record 39036 of 40000.
Wrong Type - Skipped Record 39037 of 40000.
Wrong Type - Skipped Record 39038 of 40000.
Wrong Type - Skipped Record 39039 of 40000.
Processed Record 39040 of 40000.

Wrong Type - Skipped Record 39233 of 40000.
Wrong Type - Skipped Record 39234 of 40000.
Wrong Type - Skipped Record 39235 of 40000.
Processed Record 39236 of 40000.
Processed Record 39237 of 40000.
Wrong Type - Skipped Record 39238 of 40000.
Wrong Type - Skipped Record 39239 of 40000.
Wrong Type - Skipped Record 39240 of 40000.
Wrong Type - Skipped Record 39241 of 40000.
Wrong Type - Skipped Record 39242 of 40000.
Processed Record 39243 of 40000.
Processed Record 39244 of 40000.
Wrong Type - Skipped Record 39245 of 40000.
Processed Record 39246 of 40000.
Wrong Type - Skipped Record 39247 of 40000.
Wrong Type - Skipped Record 39248 of 40000.
Wrong Type - Skipped Record 39249 of 40000.
Wrong Type - Skipped Record 39250 of 40000.
Processed Record 39251 of 40000.
Processed Record 39252 of 40000.
Wrong Type - Skipped Record 39253 of 40000.
Processed Record 39254 of 40000.
Processed Record 39255 of 40000.
Processed Record 39256 of 40000.
Wrong Type - Skipped Record 39257 of 40000.
Processed 

Processed Record 39448 of 40000.
Processed Record 39449 of 40000.
Wrong Type - Skipped Record 39450 of 40000.
Processed Record 39451 of 40000.
Processed Record 39452 of 40000.
Processed Record 39453 of 40000.
Wrong Type - Skipped Record 39454 of 40000.
Wrong Type - Skipped Record 39455 of 40000.
Wrong Type - Skipped Record 39456 of 40000.
Processed Record 39457 of 40000.
Processed Record 39458 of 40000.
Processed Record 39459 of 40000.
Processed Record 39460 of 40000.
Processed Record 39461 of 40000.
Wrong Type - Skipped Record 39462 of 40000.
Wrong Type - Skipped Record 39463 of 40000.
Wrong Type - Skipped Record 39464 of 40000.
Processed Record 39465 of 40000.
Wrong Type - Skipped Record 39466 of 40000.
Processed Record 39467 of 40000.
Processed Record 39468 of 40000.
Wrong Type - Skipped Record 39469 of 40000.
Processed Record 39470 of 40000.
Wrong Type - Skipped Record 39471 of 40000.
Processed Record 39472 of 40000.
Wrong Type - Skipped Record 39473 of 40000.
Processed Record 3947

Processed Record 39666 of 40000.
Processed Record 39667 of 40000.
Processed Record 39668 of 40000.
Wrong Type - Skipped Record 39669 of 40000.
Wrong Type - Skipped Record 39670 of 40000.
Processed Record 39671 of 40000.
Wrong Type - Skipped Record 39672 of 40000.
Processed Record 39673 of 40000.
Processed Record 39674 of 40000.
Wrong Type - Skipped Record 39675 of 40000.
Processed Record 39676 of 40000.
Wrong Type - Skipped Record 39677 of 40000.
Wrong Type - Skipped Record 39678 of 40000.
Wrong Type - Skipped Record 39679 of 40000.
Processed Record 39680 of 40000.
Wrong Type - Skipped Record 39681 of 40000.
Processed Record 39682 of 40000.
Wrong Type - Skipped Record 39683 of 40000.
Processed Record 39684 of 40000.
Wrong Type - Skipped Record 39685 of 40000.
Processed Record 39686 of 40000.
Wrong Type - Skipped Record 39687 of 40000.
Processed Record 39688 of 40000.
Processed Record 39689 of 40000.
Wrong Type - Skipped Record 39690 of 40000.
Wrong Type - Skipped Record 39691 of 40000.

Processed Record 39878 of 40000.
Processed Record 39879 of 40000.
Processed Record 39880 of 40000.
Wrong Type - Skipped Record 39881 of 40000.
Processed Record 39882 of 40000.
Processed Record 39883 of 40000.
Wrong Type - Skipped Record 39884 of 40000.
Processed Record 39885 of 40000.
Processed Record 39886 of 40000.
Wrong Type - Skipped Record 39887 of 40000.
Processed Record 39888 of 40000.
Wrong Type - Skipped Record 39889 of 40000.
Processed Record 39890 of 40000.
Processed Record 39891 of 40000.
Processed Record 39892 of 40000.
Processed Record 39893 of 40000.
Processed Record 39894 of 40000.
Processed Record 39895 of 40000.
Processed Record 39896 of 40000.
Processed Record 39897 of 40000.
Processed Record 39898 of 40000.
Processed Record 39899 of 40000.
Processed Record 39900 of 40000.
Wrong Type - Skipped Record 39901 of 40000.
Processed Record 39902 of 40000.
Wrong Type - Skipped Record 39903 of 40000.
Wrong Type - Skipped Record 39904 of 40000.
Wrong Type - Skipped Record 3990

In [13]:
print("Request Results:")
print("Success #:" + str(len(rSuccess)))
print("Skipped #:" + str(len(rFailure)))

Request Results:
Success #:21231
Skipped #:18733


In [24]:
# Create dataframe with addresses from API requests
address_df = pd.DataFrame({
    "house #": house_num,
    "street": road,
    "zipcode": postcode,
    "lat": lat,
    "lon": lon,
    "neighborhood": neighborhood,
})

# Clean up Dataframe Columns before output (Drop incomplete zipcodes, Highway streets, and Null house # or streets)
address_df = address_df[address_df['zipcode'].str.len() == 5]
address_df = address_df[address_df['zipcode'].apply(lambda x: len(str(x)) > 3)]
address_df = address_df[address_df['street'].str.contains("Highway") == False]
address_df = address_df[address_df['house #'].str.contains("NA") == False]
address_df = address_df[address_df['street'].str.contains("NA") == False]
address_df.dtypes

house #         object
street          object
zipcode         object
lat             object
lon             object
neighborhood    object
dtype: object

In [15]:
# write a CSV file of addresses
address_df.to_csv(addressList_csv, index=False, header=True)

In [ ]:
# Map out CSV with gmplot

addressList_csv = "./Data/addressList.csv"

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

gmap.scatter(addressList_csv["lat"], addressList_csv["lon"], 'red', size=20, marker=False)

gmap.draw("./Visuals/myaddressmap.html")

## Zillow API Calls using Address and Zipcode

In [ ]:
# Val create function for read in csv, check for headers?, and append csv


In [25]:
#############################
##### VALERIE'S BLOCKS #####
###########################

# Tiny sample to work with looping without exhausting API call limits
addressListTiny_csv = "./Data/addressListTiny.csv"

address_sample = pd.read_csv(addressListTiny_csv)
address_df = pd.DataFrame(address_sample)
print(len(address_df))
address_df.head()

442


,house #,street,zipcode,lat,lon,neighborhood
0,8109,Boggy Ridge Drive,78748,30.182018,-97.795088,Dittmar - Cooper
1,8507,Caspian Drive,78749,30.199637,-97.841057,Woodstone Village
2,950,Westbank Drive,78746,30.278182,-97.809151,Ledgeway
3,4208,Steck Avenue,78759,30.377557,-97.757560,Mesa Forest
4,807,Park Boulevard,78751,30.301572,-97.722516,Hyde Park


In [ ]:
####################### NO LONGER USING THIS API ########################
################# HOWEVER, CODE MAY BE USED ELSEWHERE ##################
########### OR WE MAY END UP NEEDING THIS API FOR ISD DATA ############

# pp = pprint.PrettyPrinter(indent=4)
# api = zillow.ValuationApi()
# # Insert your Zillow API key here
# zwsid = "X1-ZWz1gm14kn7d3f_2kpit"

# valuation = []
# sqft = []

# for row, home in address_df.iterrows():
#     print(row)
#     address = address_df["house #"][row] + " " + address_df["street"][row] 
#     print(address)
#     zipcode = address_df["zipcode"][row]
#     print(zipcode)
    
#     try:
#         z_deep_results = api.GetDeepSearchResults(zwsid, address, zipcode)
#         print(z_deep_results)
#         pp.pprint(deep_results.get_dict())
#         print(z_deep_results['zestimate']['amount'])
#         valuation.append(z_deep_results['zestimate']['amount'])
#         print(z_deep_results['extended_data']['finished_sqft'])
#         sqft.append(z_deep_results['extended_data']['finished_sqft'])
#     except:
#         print(f"No record found for {address}, {zipcode}.")



In [13]:
zid = []
alats = []
alons = []
addresses = []
valuation = []
sqft = []
isd = []


############## LOOPING FUNCTION FULLY OPERATIONAL ###############
###### HOWEVER, ZILLOW ONLY ALLOWS 1000 API CALLS PER DAY ######

for row, home in address_df.iterrows():
    address = str(address_df["house #"][row]) + " " + address_df["street"][row]
    addresses.append(address)
    zipcode = address_df["zipcode"][row]
    print(f"Processing {address}, {zipcode} (index {row}).")

    try:
        zillow_data = ZillowWrapper(Zyuta)
        deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
        result = GetDeepSearchResults(deep_search_response)
    except:
        print(f"No record found for {address}, {zipcode} (index {row}). Appending lists with null values")
        zid.append(None)
        alats.append(None)
        alons.append(None)
        valuation.append(None)
        sqft.append(None)
        isd.append(None)
        continue

    try:
        zillowID = result.zillow_id
        zid.append(zillowID)
    except:
        print(f"No zid found for {address}, {zipcode} (index {row}). Appending list with null values")
        zid.append(None)
        
    try:
        alat = result.latitude
        alats.append(alat)
    except:
        print(f"No alat found for {address}, {zipcode} (index {row}). Appending list with null values")
        alats.append(None)

    try:
        alon = result.longitude
        alons.append(alon)
    except:
        print(f"No alon found for {address}, {zipcode} (index {row}). Appending list with null values")
        alons.append(None)
        
    try:    
        val = int(result.zestimate_amount)
        valuation.append(val)
    except:
        print(f"No valuation found for {address}, {zipcode} (index {row}). Appending list with null values")
        valuation.append(None)

    try:
        zsqft = int(result.home_size)
        sqft.append(zsqft)
    except:
        print(f"No sqft found for {address}, {zipcode} (index {row}). Appending list with null values")
        sqft.append(None)


    try:
        zillow_data = ZillowWrapper(Zyuta)
        updated_property_details_response = zillow_data.get_updated_property_details(zid[row])
        new_result = GetUpdatedPropertyDetails(updated_property_details_response)

        try:
            zisd = new_result.school_district
            isd.append(zisd)
        except:
            isd.append(None)
            
    except:
        print(f"No updated property info for this listing (index {row}).")
        isd.append(None)


Processing 8109 Boggy Ridge Drive, 78748 (index 0).
Processing 8507 Caspian Drive, 78749 (index 1).
No updated property info for this listing (index 1).
Processing 950 Westbank Drive, 78746 (index 2).
Processing 4208 Steck Avenue, 78759 (index 3).
No record found for 4208 Steck Avenue, 78759 (index 3). Appending lists with null values
Processing 807 Park Boulevard, 78751 (index 4).
No updated property info for this listing (index 4).
Processing 10109 Slaughter Creek Drive, 78748 (index 5).
No record found for 10109 Slaughter Creek Drive, 78748 (index 5). Appending lists with null values
Processing 601 Jet Lane, 78742 (index 6).
No updated property info for this listing (index 6).
Processing 4409 Jessamine Hollow, 78731 (index 7).
No updated property info for this listing (index 7).
Processing 8800 Daffan Lane, 78724 (index 8).
No record found for 8800 Daffan Lane, 78724 (index 8). Appending lists with null values
Processing 4501 Ridge Oak Drive, 78731 (index 9).
No updated property inf

Processing 11200 Taylor Draper Lane, 78759 (index 78).
Processing 7201 RM 2222, 78730 (index 79).
No record found for 7201 RM 2222, 78730 (index 79). Appending lists with null values
Processing 9333 Brown Lane, 78754 (index 80).
No record found for 9333 Brown Lane, 78754 (index 80). Appending lists with null values
Processing 2639 Gwendolyn Lane, 78748 (index 81).
No updated property info for this listing (index 81).
Processing 11901 River Oaks Trail, 78753 (index 82).
No updated property info for this listing (index 82).
Processing 5409 Wasson Road, 78745 (index 83).
No record found for 5409 Wasson Road, 78745 (index 83). Appending lists with null values
Processing 2928 Norfolk Drive, 78745 (index 84).
Processing 9428 Parkfield Drive, 78758 (index 85).
No record found for 9428 Parkfield Drive, 78758 (index 85). Appending lists with null values
Processing 1701 Victoria Drive, 78721 (index 86).
No updated property info for this listing (index 86).
Processing 728 Barton Creek Boulevard, 

Processing 2936 Oestrick Lane, 78733 (index 153).
No record found for 2936 Oestrick Lane, 78733 (index 153). Appending lists with null values
Processing 5701 Springdale Road, 78723 (index 154).
No record found for 5701 Springdale Road, 78723 (index 154). Appending lists with null values
Processing 4820 Trail Crest Circle, 78735 (index 155).
Processing 1504 East 37th Street, 78722 (index 156).
Processing 5816 Harold Court, 78721 (index 157).
No record found for 5816 Harold Court, 78721 (index 157). Appending lists with null values
Processing 10514 Wylie Drive, 78748 (index 158).
No updated property info for this listing (index 158).
Processing 2205 Rabb Glen Street, 78704 (index 159).
Processing 6315 Pathfinder Drive, 78759 (index 160).
Processing 10001 Middle Fiskville Road, 78753 (index 161).
No record found for 10001 Middle Fiskville Road, 78753 (index 161). Appending lists with null values
Processing 4016 Pinckney Street, 78722 (index 162).
Processing 203 Canyon Rim Drive, 78746 (in

No record found for 6600 West William Cannon Drive, 78735 (index 228). Appending lists with null values
Processing 713 Kinney Avenue, 78704 (index 229).
Processing 5600 Decker Lane, 78724 (index 230).
No record found for 5600 Decker Lane, 78724 (index 230). Appending lists with null values
Processing 1707 Overhill Drive, 78721 (index 231).
No updated property info for this listing (index 231).
Processing 2412 Burleson Court, 78741 (index 232).
Processing 5634 Sedona Drive, 78759 (index 233).
Processing 8507 Bell Mountain Drive, 78730 (index 234).
No updated property info for this listing (index 234).
Processing 7164 Ridge Oak Road, 78749 (index 235).
No updated property info for this listing (index 235).
Processing 1507 Tall Shadows Drive, 78617 (index 236).
No valuation found for 1507 Tall Shadows Drive, 78617 (index 236). Appending list with null values
No sqft found for 1507 Tall Shadows Drive, 78617 (index 236). Appending list with null values
No updated property info for this list

Processing 8425 Asmara Drive, 78750 (index 302).
Processing 9020 Bluff Springs Road, 78747 (index 303).
No updated property info for this listing (index 303).
Processing 201 Edwin Lane, 78742 (index 304).
No updated property info for this listing (index 304).
Processing 2900 Susquehanna Lane, 78723 (index 305).
No updated property info for this listing (index 305).
Processing 8501 Lava Hill Road, 78744 (index 306).
No record found for 8501 Lava Hill Road, 78744 (index 306). Appending lists with null values
Processing 6201 Bolm Road, 78721 (index 307).
No record found for 6201 Bolm Road, 78721 (index 307). Appending lists with null values
Processing 2721 Lost Creek Boulevard, 78735 (index 308).
No record found for 2721 Lost Creek Boulevard, 78735 (index 308). Appending lists with null values
Processing 3703 Turkey Creek Drive, 78730 (index 309).
No updated property info for this listing (index 309).
Processing 7443 Bee Caves Road, 78746 (index 310).
No record found for 7443 Bee Caves Ro

No record found for 9383 New Airport Drive, 78719 (index 371). Appending lists with null values
Processing 6523 Twin Creek Hollow, 78750 (index 372).
No record found for 6523 Twin Creek Hollow, 78750 (index 372). Appending lists with null values
Processing 4604 Island Cove, 78731 (index 373).
No updated property info for this listing (index 373).
Processing 4702 Trail West Drive, 78735 (index 374).
No updated property info for this listing (index 374).
Processing 12030 Samsung Boulevard, 78754 (index 375).
No record found for 12030 Samsung Boulevard, 78754 (index 375). Appending lists with null values
Processing 5701 East Martin Luther King Jr Boulevard, 78721 (index 376).
No record found for 5701 East Martin Luther King Jr Boulevard, 78721 (index 376). Appending lists with null values
Processing 3103 Sweet Autumn Cove, 78735 (index 377).
No updated property info for this listing (index 377).
Processing 113 Tumbleweed Trail South, 78733 (index 378).
No record found for 113 Tumbleweed T

TypeError: must be str, not float

In [ ]:
######### ISD IS DRAWN FROM GET_UPDATED_PROPERTY_DETAILS ##########
########### WHICH HAS NOT BEEN ABLE TO RETURN VALUES  ############
############## FOR ANY PROPERTIES IN TINY SAMPLE ################
### MAY NEED TO APPROACH THESE VALUES FROM A DIFFERENT ROUTE ###

## Calculate Value per Sqft

In [14]:
valsqft = []
for row, value in enumerate(valuation):
    try:
        vsqft = round((valuation[row] / sqft[row]), 2)
        valsqft.append(vsqft)
    except:
        print("Cannot perform math with NoneType")
        valsqft.append(None)

Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform math with NoneType
Cannot perform

## Master Dataframe Creation

In [26]:
# Checking to ensure lists are appropriate lengths
print(len(zid))
print(len(alats))
print(len(alons))
print(len(addresses))
print(len(valuation))
print(len(sqft))
print(len(valsqft))
print(len(isd)) 

# Referring back to addressListTiny_csv generated dataframe for relevant info
address_df.head()

442
442
442
442
442
442
442
442


,house #,street,zipcode,lat,lon,neighborhood
0,8109,Boggy Ridge Drive,78748,30.182018,-97.795088,Dittmar - Cooper
1,8507,Caspian Drive,78749,30.199637,-97.841057,Woodstone Village
2,950,Westbank Drive,78746,30.278182,-97.809151,Ledgeway
3,4208,Steck Avenue,78759,30.377557,-97.757560,Mesa Forest
4,807,Park Boulevard,78751,30.301572,-97.722516,Hyde Park


In [36]:
# CURRENTLY ONLY SHOWING TINY SAMPLE

masterDF = pd.DataFrame({
    "Zillow ID": zid,
    "address": addresses,
    "zipcode": address_df["zipcode"],
    "alat": alats,
    "alon": alons,
    "valuation": valuation,
    "sqft": sqft,
    "value sqft": valsqft,
    "neighborhood": address_df["neighborhood"],
})
len(masterDF)

442

In [35]:
masterDFclean = masterDF.dropna(how="any", subset=["Zillow ID"])
len(masterDFclean)

277

In [37]:
# masterDF to csv
masterDFclean.to_csv(masterData_csv, index=False, header=True)
masterDFclean.head(30)

,Zillow ID,address,zipcode,alat,alon,valuation,sqft,value sqft,neighborhood
0,58316011,8109 Boggy Ridge Drive,78748,30.182015,-97.795063,262808.0,1476.0,178.05,Dittmar - Cooper
1,29498766,8507 Caspian Drive,78749,30.199658,-97.841024,310834.0,1654.0,187.93,Woodstone Village
2,2132336893,950 Westbank Drive,78746,30.278708,-97.808911,805208.0,1582.0,508.98,Ledgeway
4,29400629,807 Park Boulevard,78751,30.301584,-97.722499,583926.0,1905.0,306.52,Hyde Park
6,29462460,601 Jet Lane,78742,30.235276,-97.677214,250568.0,1015.0,246.87,NaN
7,29349645,4409 Jessamine Hollow,78731,30.348537,-97.776564,521321.0,2900.0,179.77,Cat Mountain
9,29343151,4501 Ridge Oak Drive,78731,30.329848,-97.769843,2954219.0,10778.0,274.10,Balcones Park
12,70339207,11201 Long Summer Drive,78754,30.359344,-97.640621,292348.0,3183.0,91.85,Pioneer Crossing East
13,58302468,504 East Braker Lane,78753,30.378232,-97.67872,260715.0,1264.0,206.26,Eubank Acres
14,29402786,5703 Coolbrook Drive,78724,30.304843,-97.656442,357860.0,2282.0,156.82,Las Cimas


## Crime Data

In [ ]:
# Kat's section

## School Data

In [ ]:
# Seth's section

## Heat Mapper

In [ ]:
# Troy's section


gmaps.configure(api_key=google_API_Key)

In [ ]:
# This cell creates a test masterData_df by pulling in Yuta's address file and adds a column as a testm "value to map"
# This cell can be deleted as soon as there is a master data file that includes a property value column or some other value to plot
# The last digit of the zipcode is used as a value that will vary by area and a random number between 0 and 1 is added to create variation in the weights

masterData_df = pd.read_csv(addressList_csv)
zips = masterData_df["zipcode"]
valueToMap = []

for zip in zips:
    lastDigit = zip[-1:]
#    print(last2Digits)
    valueToMap.append(int(lastDigit) + random.uniform(0.0,1.0))
    
masterData_df["valueToMap"] = valueToMap
masterData_df.head()

In [ ]:
# This cell uses gmaps library to create a google heat map from the data in a master data file.
# The masterData csv file is taken as input
# The lat and lon columns are taken as the coordinates for hte heatmap 
# The user specified column is taken as the weighting valies fo each coordinate point

df = masterData_df
columnToMap = 'valueToMap'
max_intensity = df[columnToMap].max()

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
fig.add_layer(heatmap_layer)
fig

In [ ]:
# this is a function version of the cell above
# the function takes columnToMap as the weights for the points defined by 'lat' and 'lon' columns in the dataframe
# the dataframe can be included as a parameter, if it is not included masterData_df is assumed

def heatMapper(columnToMap, df = masterData_df):
    
    max_intensity = df[columnToMap].max()
    
    fig = gmaps.figure()
    heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
    fig.add_layer(heatmap_layer)

    return;

In [ ]:
heatMapper(columnToMap = 'valueToMap')
fig